In [143]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter



In [144]:
dfx= pd.read_excel('resultados_elecciones_presidenciales_ce_1989_2017_Chile.xlsx')

In [145]:
print(dfx.head())
print(dfx.columns)
print(dfx.shape)
#df.describe()
print(dfx.info())
#df.describe(percentiles=[0.25, 0.5, 0.75])
dfx.describe(include='all')

  Tipo de Elección       Cargo Fecha de Elección  Año de Elección  \
0     PRESIDENCIAL  PRESIDENTE        1989-12-14             1989   
1     PRESIDENCIAL  PRESIDENTE        1989-12-14             1989   
2     PRESIDENCIAL  PRESIDENTE        1989-12-14             1989   
3     PRESIDENCIAL  PRESIDENTE        1989-12-14             1989   
4     PRESIDENCIAL  PRESIDENTE        1989-12-14             1989   

   Inicio de Período  Fin de Período      Período Votación Presidencial  \
0               1990            1994  1990 a 1994        UNICA VOTACIÓN   
1               1990            1994  1990 a 1994        UNICA VOTACIÓN   
2               1990            1994  1990 a 1994        UNICA VOTACIÓN   
3               1990            1994  1990 a 1994        UNICA VOTACIÓN   
4               1990            1994  1990 a 1994        UNICA VOTACIÓN   

   Id Región       Región  ...  Sexo Mesa               Candidato (a)  \
0          1  DE TARAPACA  ...     HOMBRE  FRANCISCO JAVIER E

,Tipo de Elección,Cargo,Fecha de Elección,Año de Elección,Inicio de Período,Fin de Período,Período,Votación Presidencial,Id Región,Región,...,Sexo Mesa,Candidato (a),Nombres,Apellido Paterno,Apellido Materno,Sexo,Electo(a),Partido,Sigla Partido,Votos Totales
count,64366,64366,64366,64366.000000,64366.000000,64366.000000,64366,64366,64366.000000,64366,...,64366,64366,42748,42748,42748,42748,15882,42748,41620,64366.000000
unique,1,1,NaN,NaN,NaN,NaN,7,3,NaN,17,...,3,40,33,30,30,2,3,22,19,NaN
top,PRESIDENCIAL,PRESIDENTE,NaN,NaN,NaN,NaN,2000 a 2006,PRIMERA VOTACIÓN,NaN,DEL BIOBIO,...,MUJER,VOTOS NULOS,SEBASTIAN,PIÑERA,ECHENIQUE,HOMBRE,2ª Votación,INDEPENDIENTE,INDEP,NaN
freq,64366,64366,NaN,NaN,NaN,NaN,12200,34078,NaN,9724,...,23525,8413,5719,6695,6695,34393,8894,9064,6792,NaN
mean,NaN,NaN,2005-08-06 23:22:14.157785216,2004.847124,2005.645030,2010.266725,NaN,NaN,8.266274,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1306.059239
min,NaN,NaN,1989-12-14 00:00:00,1989.000000,1990.000000,1994.000000,NaN,NaN,1.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
25%,NaN,NaN,1999-12-12 00:00:00,1999.000000,2000.000000,2006.000000,NaN,NaN,6.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.000000
50%,NaN,NaN,2006-01-15 00:00:00,2006.000000,2006.000000,2010.000000,NaN,NaN,8.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,129.000000
75%,NaN,NaN,2013-11-17 00:00:00,2013.000000,2014.000000,2018.000000,NaN,NaN,10.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,795.000000
max,NaN,NaN,2017-12-17 00:00:00,2017.000000,2018.000000,2022.000000,NaN,NaN,15.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80163.000000


In [146]:
df2= dfx[['Año de Elección','Id Región','Región','Candidato (a)','Partido','Sigla Partido','Votos Totales']]
df3= df2[['Candidato (a)','Partido']].groupby(['Candidato (a)','Partido']).sum().reset_index()
#Se exporta lista de candidatos para ingreso manual de la tendencia politica
df3.to_excel('./tmp/Candidatos_1989-2017.xlsx')

In [147]:
df4=pd.read_excel('./tmp/Candidatos_1989-2017_con_tendencia.xlsx')   
df4.head()
df2= df2.merge(df4, on='Candidato (a)', how='left')
df2.loc[df2['Candidato (a)'].str.contains('VOTOS EN BLANCO', case=False, na=False), 'Tendencia Politica'] = 'VB-VN'
df2.loc[df2['Candidato (a)'].str.contains('VOTOS NULOS', case=False, na=False), 'Tendencia Politica'] = 'VB-VN'

df2.to_excel('./tmp/Candidatos_1989-2017_ajustado.xlsx')

In [149]:
df4 = df2[['Año de Elección','Tendencia Politica','Votos Totales']].groupby(['Año de Elección','Tendencia Politica']).sum().reset_index()
df4['Total por Año'] = df4.groupby('Año de Elección')['Votos Totales'].transform('sum')
df4['Porcentaje'] = (df4['Votos Totales'] / df4['Total por Año']) * 100

pivot_df4 = df4.pivot_table(index='Año de Elección', columns='Tendencia Politica', 
                            values='Porcentaje', aggfunc='sum')

# Restablecer el índice para que 'Año de Elección' quede como columna
pivot_df4 = pivot_df4.reset_index(drop=False)

# Asegurarse de que las columnas sigan el formato deseado: 'Año', 'Derecha', 'Centro', 'Izquierda', 'VB-VN'
column_order = ['Año de Elección', 'Derecha', 'Centro', 'Izquierda', 'VB-VN']
pivot_df4 = pivot_df4.reindex(columns=column_order)

# Rellenar cualquier valor nulo con 0
pivot_df4 = pivot_df4.fillna(0)

df=pivot_df4
df.set_index('Año de Elección', inplace=True)
tendencias = ['Centro', 'Derecha', 'Izquierda', 'VB-VN']

# Función para calcular la matriz de transición entre dos años (sin cambios)
def calcular_transicion(df, año_anterior, año_actual):
    porcentajes_anterior = df.loc[año_anterior, tendencias]
    porcentajes_actual = df.loc[año_actual, tendencias]
    cambios = porcentajes_actual - porcentajes_anterior

    # Identificar ganadores y perdedores
    ganadores = cambios[cambios > 0]
    perdedores = cambios[cambios < 0] * -1  # Convertir a valores positivos

    # Sumar total de ganancias y pérdidas
    total_ganancias = ganadores.sum()
    total_perdidas = perdedores.sum()

    # Crear matriz de transición vacía
    transicion = pd.DataFrame(0.0, index=tendencias, columns=tendencias)

    # Si hay perdedores y ganadores
    if total_ganancias > 0 and total_perdidas > 0:
        for perdedor in perdedores.index:
            for ganador in ganadores.index:
                # Proporción de la pérdida del perdedor que va al ganador
                transicion.loc[perdedor, ganador] = perdedores[perdedor] * (ganadores[ganador] / total_ganancias)
    # Si no hay ganadores, los votos perdidos van a VB_VN
    elif total_perdidas > 0:
        for perdedor in perdedores.index:
            transicion.loc[perdedor, 'VB-VN'] += perdedores[perdedor]

    # Los que mantienen o aumentan mantienen sus votos anteriores
    for tendencia in tendencias:
        if cambios[tendencia] >= 0:
            transicion.loc[tendencia, tendencia] += min(porcentajes_anterior[tendencia], porcentajes_actual[tendencia])
        else:
            transicion.loc[tendencia, tendencia] += porcentajes_actual[tendencia]

    return transicion

# Listas para almacenar los datos necesarios para el gráfico Sankey
source_labels_list = []
target_labels_list = []
values_list = []

# Procesar cada par de años para construir las listas de source, target y values
for idx in range(len(df.index) - 1):
    año_anterior = df.index[idx]
    año_actual = df.index[idx + 1]
    transicion = calcular_transicion(df, año_anterior, año_actual)

    # Excluir columnas y filas adicionales
    matriz_sin_totales = transicion.loc[tendencias, tendencias]

    # Construir las etiquetas de origen y destino, y los valores
    for i, tendencia_origen in enumerate(tendencias):
        for j, tendencia_destino in enumerate(tendencias):
            source_label = f"{año_anterior}-{tendencia_origen}"
            target_label = f"{año_actual}-{tendencia_destino}"
            value = matriz_sin_totales.iloc[i, j]

            # Solo agregamos conexiones con valor positivo
            if value > 0:
                source_labels_list.append(source_label)
                target_labels_list.append(target_label)
                values_list.append(value)

# Crear una lista de etiquetas únicas
labels = list(set(source_labels_list + target_labels_list))

# Crear un diccionario para mapear etiquetas a índices
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Mapear las etiquetas de origen y destino a índices
source_indices = [label_to_index[label] for label in source_labels_list]
target_indices = [label_to_index[label] for label in target_labels_list]


# Función para convertir hex a RGBA
def hex_to_rgba(hex_color, alpha=1.0):
    hex_color = hex_color.lstrip('#')
    r, g, b = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
    return f'rgba({r}, {g}, {b}, {alpha})'

# Mapa de colores para cada tendencia
color_map = {
    'Centro': '#1f77b4',      # Azul
    'Derecha': '#ff7f0e',     # Naranja
    'Izquierda': '#2ca02c',   # Verde
    'VB_VN': '#d62728'        # Rojo
}

# Generar lista de colores RGBA para los nodos con opacidad completa
node_colors = []
for label in labels:
    tendencia = label.split('-')[1]
    hex_color = color_map.get(tendencia, '#7f7f7f')  # Gris por defecto
    rgba_color = hex_to_rgba(hex_color, alpha=1.0)    # Opacidad completa para nodos
    node_colors.append(rgba_color)

# Generar lista de colores RGBA para los enlaces con menor opacidad
link_colors = []
for src_idx in source_indices:
    # Obtener la tendencia del nodo de origen
    tendencia = labels[src_idx].split('-')[1]
    hex_color = color_map.get(tendencia, '#7f7f7f')  # Gris por defecto
    # Convertir a RGBA con opacidad reducida (por ejemplo, 0.3)
    rgba_color = hex_to_rgba(hex_color, alpha=0.3)
    link_colors.append(rgba_color)









# Crear el gráfico Sankey
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=50,# Controlar el espacio entre los nodos
        thickness=30, # Controlar el grosor de las líneas
        line=dict(color="black", width=0.5),
        label=labels,
        color=node_colors
    ),
    link=dict(
        source=source_indices,
        target=target_indices,
        value=values_list,
        color=link_colors
    ))])

fig.update_layout(
    title_text="Transiciones de Tendencias Políticas",
    font_size=10,
    width=1400,
    height=900
)
fig.show()
